In [1]:
import cobra,rba,lxml
from lxml import etree
import pandas as pd

In [2]:
mModel = cobra.io.load_json_model('./input/model/y834_hvd_v3_rba.json')

Compartment

In [3]:
modelET = etree.Element('RBAMetabolism')

elems = ['listOfCompartments', 'listOfSpecies', 'listOfReactions']
for e in elems:
    modelET.append(etree.Element(e))

In [4]:
comps = sorted(mModel.compartments.keys())
compET = modelET.find('listOfCompartments')
for comp in comps:
    compET.append(etree.Element('compartment', attrib={'id':'PART-' + comp}))

Metabolites

In [5]:
mets = sorted([met.id for met in mModel.metabolites])
metET = modelET.find('listOfSpecies')
for metid in mets:
    met = mModel.metabolites.get_by_id(metid)
    comp = met.compartment
    bcond = 'true' if comp == 'e' else 'false'
    att_dict = {'id':'MET-'+met.id, 'boundaryCondition':bcond}
    metET.append(etree.Element('species', attrib=att_dict))

Reactions

In [6]:
df_rxns = pd.read_excel('./curation/enz_stoich_curation.xlsx')
df_rxns.index = df_rxns.id.to_list()

In [7]:
#rxns = sorted([rxn.id for rxn in mModel.reactions])
rxnET = modelET.find('listOfReactions')

In [8]:
for i in df_rxns.index:
    rxnentry = df_rxns.id[i]
    rxnid = df_rxns.rxn_src[i]
    rxn = mModel.reactions.get_by_id(rxnid)
    #rev = 'true' if rxn.reversibility else 'false'
    rev = 'false'
    att_dict = {'id':rxnentry, 'reversible':rev}
    rxnE = etree.SubElement(rxnET, 'reaction', attrib=att_dict)
    
    src_reactants = sorted([met.id for met in rxn.reactants])
    src_products = sorted([met.id for met in rxn.products])
    side_mapper = {
        'FWD': (src_reactants, src_products, 1),
        'REV': (src_products, src_reactants, -1)
    }
    
    x = rxnentry.split('-')[1]
    x = x.split('_')[-1]
    reactants,products,sign = side_mapper[x]
    
    rxnE_reactants = etree.SubElement(rxnE, 'listOfReactants')
    for metid in reactants:
        met = mModel.metabolites.get_by_id(metid)
        coeff = sign*rxn.metabolites[met]
        att_dict = {'species':'MET-'+met.id, 'stoichiometry':str(coeff)}
        rxnE_reactants.append(etree.Element('speciesReference', attrib=att_dict))
        
    rxnE_products = etree.SubElement(rxnE, 'listOfProducts')
    for metid in products:
        met = mModel.metabolites.get_by_id(metid)
        coeff = sign*rxn.metabolites[met]
        att_dict = {'species':'MET-'+met.id, 'stoichiometry':str(coeff)}
        rxnE_products.append(etree.Element('speciesReference', attrib=att_dict))

In [9]:
with open('./rbaModel/metabolism.xml', 'wb') as f:
    f.write(etree.tostring(modelET, pretty_print=True))